In [22]:
import math
import os.path as osp
from tqdm.auto import tqdm
import numpy as np
import wandb

import torch
from torch.nn import BCEWithLogitsLoss, Conv1d, MaxPool1d, ModuleList
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, MLP, global_sort_pool
from torch_geometric.loader import DataLoader

from dataset_processing import RNADataset


In [23]:
dataset = RNADataset(root="./data/")
dataset = dataset.shuffle()
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]

train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)


In [5]:
def del_diags(y):
    main_diag = torch.ones([1, y.size(0)])
    diag1 = torch.ones([1, y.size(0) - 1])
    return y * (torch.diag_embed(main_diag).to(device) + 
            torch.diag_embed(diag1, offset=1).to(device) + 
            torch.diag_embed(diag1, offset=-1).to(device) - 1) * (-1)

In [6]:
def precision(y_pred, y_true):
    y_pred = del_diags(y_pred)
    y_pred[(y_pred > 0.5)] = 1
    y_pred[(y_pred <= 0.5)] = 0 
    
    tp = torch.sum(y_pred * y_true)
    fp = torch.sum((1 - y_true) * y_pred)
    
    return tp / (tp + fp + epsilon)

def recall(y_pred, y_true):
    y_pred = del_diags(y_pred)
    y_pred[(y_pred > 0.5)] = 1
    y_pred[(y_pred <= 0.5)] = 0
    
    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    
    return tp / (tp + fn + epsilon)

def f1_loss(y_pred, y_true):
    y_pred = del_diags(y_pred)

    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    fp = torch.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

#     k1 = 1 - torch.abs(precision - recall)
#     k2 = 1 - torch.abs(K.mean(precision) - K.mean(recall))
    #calculate upgraded f1 score
    f1 = 2 * precision * recall / (precision + recall + epsilon)
#     tw = K.sum(K.cast(y_true * y_pred, ’float32’), axis=[1, 2, 3])
#     fw = K.sum(K.cast((1 - y_true) * y_pred, ’float32’), axis=[1, 2, 3])
#     fb = K.sum(K.cast(y_true * (1 - y_pred), ’float32’), axis=[1, 2, 3])
    return 1 - f1


In [83]:
class DGCNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, GNN=GCNConv, k=0.6):
        super().__init__()
        if k < 1:  # Transform percentile to number.
            num_nodes = sorted([data.num_nodes for data in train_data])
            k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
            k = max(10, k)
        self.k = int(k)

        self.convs = ModuleList()
        self.convs.append(GNN(dataset.num_features, hidden_channels))
        for i in range(0, num_layers - 1):
            self.convs.append(GNN(hidden_channels, hidden_channels))
        self.convs.append(GNN(hidden_channels, 1))

        conv1d_channels = [16, 32]
        total_latent_dim = hidden_channels * num_layers + 1
        conv1d_kws = [total_latent_dim, 5]
        self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
                            conv1d_kws[0])
        self.maxpool1d = MaxPool1d(2, 2)
        self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
                            conv1d_kws[1], 1)
        dense_dim = int((self.k - 2) / 2 + 1)
        dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
        self.mlp = MLP([dense_dim, 128, 1], dropout=0.5, norm=None)

    def forward(self, x, edge_index, batch):
        xs = [x]
        for conv in self.convs:
            xs += [conv(xs[-1], edge_index).tanh()]
        x = torch.cat(xs[1:], dim=-1)

        # Global pooling.
#         x = global_sort_pool(x, batch, self.k)
#         x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
#         x = self.conv1(x).relu()
#         x = self.maxpool1d(x)
#         x = self.conv2(x).relu()
#         x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

#         return self.mlp(x)
        return (x @ x.t()).sigmoid()

In [84]:
hidden_channels = 128
out_channels = 150
num_layers = 4
num_features = dataset.num_features


dr = 0.2
lr = 0.0001
epochs = 500

model = DGCNN(hidden_channels=32, num_layers=3)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_data = list(map(lambda x: x.to(device), train_data))
val_data = list(map(lambda x: x.to(device), val_data))
print(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# criterion = torch.nn.BCELoss()
# criterion = torch.nn.BCEWithLogitsLoss()
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.KLDivLoss()

# def RMSELoss(y_pred, y_true):
#     return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

criterion = f1_loss

cuda


In [26]:
run_name = "DGCNN_" + str(num_layers) + "_" + str(epochs) + "_" + str(lr) + "_" + "Adam_" + str(out_channels)


wandb.init(
    # set the wandb project where this run will be logged
    project="secondary_structure_prediction1",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "DGCNN",
    "epochs": epochs,
    "optimizer": "Adam",
    "out_channels": out_channels,
    "loss": "CELoss",
    "train:val:test": "655:218:218"
    },
    name=run_name
)

epsilon = 1e-10

wandb: Currently logged in as: chi-vinny0702. Use `wandb login --relogin` to force relogin


In [85]:
def train():
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        train_recall = []
        train_precision = []
#         for g in tqdm(train_data, ncols=100):
        for g in tqdm(train_dataloader, ncols=100):
            g.to(device)
            optimizer.zero_grad()

            out = model(g.x, g.edge_index, g.batch)
            y_true = g.edge_label_index
            loss = criterion(out, y_true.to(torch.float32))            
            
            loss.backward()
            optimizer.step()
                        
            train_loss.append(loss.item())
            train_precision.append(precision(out, y_true).item())
            train_recall.append(recall(out, y_true).item())
            
        train_prec = np.mean(train_precision)
        train_rec = np.mean(train_recall)
        train_f1 = (2 * train_prec * train_rec) / (train_prec + train_rec)
        print(f'Epoch: {epoch:03d}, loss: {np.mean(train_loss)}, f1: {train_f1}, precision: {train_prec}, recall: {train_rec}')
        
        val_loss = []
        val_recall = []
        val_precision = []
        with torch.no_grad():
#             for g in tqdm(val_data, ncols=100):
            for g in tqdm(val_dataloader, ncols=100):

                g.to(device)
                out = model(g.x, g.edge_index, g.batch)
        
                y_true = g.edge_label_index
                loss = criterion(out, y_true.to(torch.float32))
                                
                val_loss.append(loss.item())
                
#                 out = out.sigmoid()
                
                val_precision.append(precision(out, y_true).item())
                val_recall.append(recall(out, y_true).item())
 
            
            prec = np.mean(val_precision)
            rec = np.mean(val_recall)
            f1 = (2 * prec * rec) / (prec + rec)
            print(f'val_loss: {np.mean(val_loss)}, val_f1: {f1}, val_precision: {prec}, val_recall: {rec}')
              
            wandb.log({"train_loss": np.mean(train_loss), "train_f1": train_f1, "train_precision": train_prec, 
                       "train_recall": train_rec,
                       "val_loss": np.mean(val_loss), "val_f1": f1, "val_precision": prec, "val_recall": rec})
#     wandb.finish()

In [77]:
train_data[0]

Data(x=[72, 4], edge_index=[2, 378], edge_label_index=[72, 72])

In [86]:
train()
torch.save(model, "./models1/" + run_name + ".pt")

100%|████████████████████████████████████████████████████████████| 655/655 [00:05<00:00, 122.81it/s]


Epoch: 001, loss: 0.9585552319315553, f1: 0.04421297741962869, precision: 0.02263992811012871, recall: 0.9382308747022207


100%|████████████████████████████████████████████████████████████| 218/218 [00:01<00:00, 206.54it/s]


val_loss: 0.9567201517590689, val_f1: 0.04918989755258823, val_precision: 0.02529288530819665, val_recall: 0.8913091522291166


100%|████████████████████████████████████████████████████████████| 655/655 [00:05<00:00, 113.59it/s]


...

val_loss: 0.9491820111187226, val_f1: 0.05242147140057856, val_precision: 0.027069451264960163, val_recall: 0.8262459210572987


100%|████████████████████████████████████████████████████████████| 655/655 [00:05<00:00, 123.96it/s]


Epoch: 026, loss: 0.9450709595935035, f1: 0.057072415362430504, precision: 0.029556692852318742, recall: 0.8265048326881788


 58%|██████████████████████████████████▋                         | 126/218 [00:00<00:00, 188.75it/s]


KeyboardInterrupt: 

In [ ]:
run_name = "GatedGCN_" + str(num_layers) + "_" + "140" + "_" + str(lr) + "_" + "Adam_" + str(out_channels)


torch.save(model, "./models/" + run_name + ".pt")